In [1]:
pip install icrawler


Note: you may need to restart the kernel to use updated packages.


In [2]:
vimport requests
import os

# Your Unsplash Access Key
ACCESS_KEY = "H1IVJhiehu6CL6hTwubD6biPzyvCz8F2u8qW-iadqfI"

# Function to download images from Unsplash
def download_images_unsplash(query, output_dir, num_images=100):
    """
    Download images from Unsplash API.
    Args:
        query (str): Search query for the images.
        output_dir (str): Directory to save the images.
        num_images (int): Total number of images to download.
    """
    os.makedirs(output_dir, exist_ok=True)  # Ensure the directory exists
    url = "https://api.unsplash.com/search/photos"
    
    headers = {
        "Authorization": f"Client-ID {ACCESS_KEY}"
    }
    
    total_downloaded = 0
    page = 1
    
    while total_downloaded < num_images:
        params = {
            "query": query,
            "page": page,
            "per_page": 30  # Unsplash returns a max of 30 images per page
        }
        
        response = requests.get(url, headers=headers, params=params)
        
        if response.status_code != 200:
            print(f"Failed to fetch images: {response.status_code}")
            print("Response:", response.text)  # Print the raw response content
            break
        
        try:
            data = response.json()  # Attempt to parse the response as JSON
        except requests.exceptions.JSONDecodeError:
            print("Failed to decode JSON from the response.")
            print("Response content:", response.text)  # Log the raw response
            break
        
        images = data.get("results", [])
        
        if not images:
            print("No more images found.")
            break
        
        for image in images:
            if total_downloaded >= num_images:
                break
            
            image_url = image["urls"]["full"]  # Full-resolution image URL
            image_id = image["id"]  # Unique image ID
            
            image_response = requests.get(image_url)
            
            if image_response.status_code == 200:
                # Ensure the directory exists before saving
                image_path = os.path.join(output_dir, f"{image_id}.jpg")
                os.makedirs(os.path.dirname(image_path), exist_ok=True)  # Create subdirectories if necessary
                
                with open(image_path, "wb") as file:
                    file.write(image_response.content)
                total_downloaded += 1
                print(f"Downloaded {total_downloaded}/{num_images}: {image_path}")
            else:
                print(f"Failed to download image {image_id}")
        
        page += 1  # Move to the next page

# Dataset with species and subspecies
dataset = {
    "Cat": ["Bengal Cat", "Persian Cat", "Maine Coon Cat", "Munchkin Cat", "Sphynx Cat"],
    "Bird": ["Bald Eagle", "Parrot", "Penguin", "Flamingo", "Owl"],
    "Whale": ["Blue Whale", "Humpback Whale", "Beluga Whale", "Orca", "Gray Whale"],
    "Fox": ["Red Fox", "Arctic Fox", "Fennec Fox", "Gray Fox", "Silver Fox"],
    "Bear": ["Polar Bear", "Grizzly Bear", "Panda Bear", "Black Bear", "Sun Bear"],
    "Dog": ["Golden Retriever", "German Shepherd", "Poodle", "Bulldog", "Siberian Husky"],
    "Penguin": ["Emperor Penguin", "King Penguin", "Adelie Penguin", "Chinstrap Penguin", "Gentoo Penguin"],
    "Elephant": ["African Elephant", "Asian Elephant", "Forest Elephant", "Savanna Elephant", "Sri Lankan Elephant"]
}

# Base directory for saving images in Google Drive
output_dir_base = "/content/drive/MyDrive/Unsplash_Images"

# Number of images per query
num_images = 50  # Change this to the total number of images you want per query

# Download images for each species and subspecies
for species, subspecies_list in dataset.items():
    for subspecies in subspecies_list:
        output_dir = os.path.join(output_dir_base, species, subspecies.replace(" ", "_"))  # Create subfolder for each species and subspecies
        download_images_unsplash(subspecies, output_dir, num_images)


ModuleNotFoundError: No module named 'google'